In [1]:
import numpy as np
import sys
import os
import pickle
import torch
sys.path.append('/usr0/home/yuncheng/MultiBench')
sys.path.append('/usr0/home/yuncheng/MultiBench/synthetic')
from supervised_learning import test
from ensemble import test as test_ensemble
from get_data import get_dataloader
from rus import *
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

#### dataset clustering

In [3]:
def clustering(X, pca=False, n_clusters=20, n_components=5):
  X = np.nan_to_num(X)
  if len(X.shape) > 2:
    X = X.reshape(X.shape[0],-1)
  if pca:
    # print(np.any(np.isnan(X)), np.all(np.isfinite(X)))
    X = normalize(X)
    X = PCA(n_components=n_components).fit_transform(X)
  kmeans = KMeans(n_clusters=n_clusters).fit(X)
  return kmeans.labels_, X

In [3]:
for i in range(1, 6):
    data_dir = '/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_synthetic{}.pickle'.format(i)
    dataset = pd.read_pickle(data_dir)
    n_components = 2
    data_cluster = dict()
    for split in ['valid2', 'test']:
        data_cluster[split] = dict()
        data = dataset[split]
        kmeans_0, data_0 = clustering(data['0'], pca=True, n_components=n_components, n_clusters=20)
        data_cluster[split]['0'] = kmeans_0.reshape(-1,1)
        kmeans_1, data_1 = clustering(data['1'], pca=True, n_components=n_components, n_clusters=20)
        data_cluster[split]['1'] = kmeans_1.reshape(-1,1)
        data_cluster[split]['label'] = data['label']
    with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_synthetic{}_cluster.pickle'.format(i), 'wb') as f:
        pickle.dump(data_cluster, f)

#### get dataset measures

In [4]:
results = dict()
for i in range(1,6):
    with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_synthetic{}_cluster.pickle'.format(i), 'rb') as f:
        dataset = pickle.load(f)
    print('synthetic', i)
    data = (dataset['test']['0'], dataset['test']['1'], dataset['test']['label'])
    P, maps = convert_data_to_distribution(*data)
    result = get_measure(P)
    results['synthetic{}'.format(i)] = result
    print()

with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/datasets.pickle', 'wb') as f:
    pickle.dump(results, f)

synthetic 1
Redundancy 0.042766259538634416
Unique 0.02596364321132929
Unique 2.17157384344753e-15
Synergy 0.10614122901494483

synthetic 2
Redundancy 0.012672691701037832
Unique 2.8259215299914983e-05
Unique 0.0061237290331549
Synergy 0.05926624624675512

synthetic 3
Redundancy 0.027742116209452286
Unique 0.025980735076023997
Unique 1.4684105273299479e-15
Synergy 0.06713126114497113

synthetic 4
Redundancy 0.06557045989622268
Unique 2.1518739480906317e-15
Unique 0.056037809617245915
Synergy 0.053704370671353585

synthetic 5
Redundancy 0.03943597142606828
Unique 0.043663108779810396
Unique 4.6119695336763056e-15
Synergy 0.06589532206160133



#### post-process model predictions

In [8]:
METHODS = ['additive', 'agree', 'align', 'early_fusion', 'elem', 'mfm', 'mi', 'mult', 'outer', 'lower']
MEASURES = ['redundancy', 'unique1', 'unique2', 'synergy']
SETTINGS = ['synthetic{}'.format(i) for i in range(1,6)] + ['maps', 'maps2']

In [6]:
if os.path.isfile('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/results.pickle'):
    with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/results.pickle', 'rb') as f:
        results = pickle.load(f)
else:
    results = dict()
for method in METHODS:
    results[method] = dict()
    for SETTING in SETTINGS:
        print(SETTING)
        data_path = '/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_{}.pickle'.format(SETTING)
        saved_model = '/usr0/home/yuncheng/MultiBench/synthetic/model_selection/{}/{}_{}_best.pt'.format(SETTING, SETTING, method)
        saved_cluster = '/usr0/home/yuncheng/MultiBench/synthetic/model_selection/{}/{}_{}_cluster.pickle'.format(SETTING, SETTING, method)
        _, _, _, testdata = get_dataloader(path=data_path, keys=['0','1','label'], modalities=[1,1], batch_size=128, num_workers=4)
        model = torch.load(saved_model).cuda()
        num_params = sum([p.numel() for p in model.parameters()])
        if method in ['additive', 'agree', 'align']:
            acc = test_ensemble(model, testdata, no_robust=True, criterion=torch.nn.CrossEntropyLoss(), save_preds=saved_cluster)
        else:
            acc = test(model, testdata, no_robust=True, criterion=torch.nn.CrossEntropyLoss(), save_preds=saved_cluster)
        with open(saved_cluster, 'rb') as f:
            preds = pickle.load(f)
        with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_{}_cluster.pickle'.format(SETTING), 'rb') as f:
            cluster = pickle.load(f)
        pred_results = (cluster['test']['0'], cluster['test']['1'], preds.reshape(-1,1))
        P, maps = convert_data_to_distribution(*pred_results)
        results[method][SETTING] = get_measure(P)
        results[method][SETTING]['acc'] = acc
        results[method][SETTING]['params'] = num_params
        print()

with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/results.pickle', 'wb') as f:
    pickle.dump(results, f)

synthetic1
Train data: 14000
Valid data 1: 2100
Valid data 2: 2100
Test data: 1800
loss: tensor(0.8502, device='cuda:0')
acc: 0.6283333333333333
Redundancy 0.2707825379424073
Unique 0.16334202068341028
Unique 3.266602334664933e-07
Synergy 0.09782503764897238

synthetic2
Train data: 14000
Valid data 1: 2100
Valid data 2: 2100
Test data: 1800


/usr0/home/yuncheng/miniconda3/envs/multibench/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


loss: tensor(0.8568, device='cuda:0')
acc: 0.5616666666666666
Redundancy 0.2539995938687824
Unique 0.23269513344692727
Unique 2.354576132383619e-07
Synergy 0.05884751091946233

synthetic3
Train data: 14000
Valid data 1: 2100
Valid data 2: 2100
Test data: 1800


/usr0/home/yuncheng/miniconda3/envs/multibench/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


loss: tensor(1.0994, device='cuda:0')
acc: 0.4261111111111111
Redundancy 0.06405922085532612
Unique 0.23564387158508435
Unique 1.0616968579028281e-08
Synergy 0.14250508125130634

synthetic4
Train data: 14000
Valid data 1: 2100
Valid data 2: 2100
Test data: 1800


...

/usr0/home/yuncheng/miniconda3/envs/multibench/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


acc: 0.6611111111111111
Redundancy 0.11364093525405028
Unique 0.2545367324290119
Unique 1.0846433085366024e-10
Synergy 0.07272122364189781

maps
Train data: 672
Valid data: 84
Test data: 88


/usr0/home/yuncheng/miniconda3/envs/multibench/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


acc: 0.4090909090909091
Redundancy 0.1590486890256423
Unique 0.38409604269174047
Unique 2.344669587668299e-07
Synergy 0.20995459666534433



/usr0/home/yuncheng/miniconda3/envs/multibench/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


#### maps

In [5]:
with open ('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/maps.pkl', 'rb') as f:
    maps = pickle.load(f)
N = len(maps['bow'])
print(N, maps.keys())
data = dict()
data['train'] = dict()
data['valid'] = dict()
data['test'] = dict()
for i, f in enumerate(['bow', 'typing']):
    data['train'][str(i)] = np.array(maps[f][:N//10*8], dtype=float)
    data['valid'][str(i)] = np.array(maps[f][N//10*8:N//10*9], dtype=float)
    data['test'][str(i)] = np.array(maps[f][N//10*9:], dtype=float)
data['train']['label'] = maps['label'][:N//10*8]
data['valid']['label'] = maps['label'][N//10*8:N//10*9]
data['test']['label'] = maps['label'][N//10*9:]
with open ('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_maps2.pickle', 'wb') as f:
    pickle.dump(data, f)

844 dict_keys(['bow', 'apps', 'typing', 'text_char', 'epoch_diff', 'word_epoch_diff', 'bow_onehot', 'apps_onehot', 'typing_onehot', 'label'])


In [4]:
with open ('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_maps.pickle', 'rb') as f:
    dataset = pickle.load(f)
print(dataset.keys())
print(dataset['train']['0'].shape, dataset['train']['1'].shape)
print(np.unique(dataset['train']['label']))

dict_keys(['train', 'valid', 'test'])
(672, 1000) (672, 137)
[0 1 2]


In [8]:
n_components = 2
data_cluster = dict()
for split in ['test']:
    data_cluster[split] = dict()
    data = dataset[split]
    kmeans_0, data_0 = clustering(data['0'], pca=True, n_components=n_components, n_clusters=20)
    data_cluster[split]['0'] = kmeans_0.reshape(-1,1)
    kmeans_1, data_1 = clustering(data['1'], pca=True, n_components=n_components, n_clusters=20)
    data_cluster[split]['1'] = kmeans_1.reshape(-1,1)
    data_cluster[split]['label'] = data['label']
with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_maps2_cluster.pickle', 'wb') as f:
    pickle.dump(data_cluster, f)


In [13]:
if os.path.isfile('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/datasets.pickle'):
    with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/datasets.pickle', 'rb') as f:
        results = pickle.load(f)
else:
    results = dict()
with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_maps2_cluster.pickle', 'rb') as f:
    dataset = pickle.load(f)
print('maps')
data = (dataset['test']['0'], dataset['test']['1'], dataset['test']['label'])
P, maps = convert_data_to_distribution(*data)
result = get_measure(P)
results['maps2'] = result     

with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/datasets.pickle', 'wb') as f:
    pickle.dump(results, f)

maps
Redundancy 0.12120173624349637
Unique 0.004155439963112993
Unique 0.03654196259113951
Synergy 0.40223033982849343


/usr0/home/yuncheng/miniconda3/envs/multibench/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [15]:
if os.path.isfile('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/results.pickle'):
    with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/results.pickle', 'rb') as f:
        results = pickle.load(f)
else:
    results = dict()
for method in ['mi', 'mult']:
    for SETTING in ['maps2']:
        print(SETTING)
        data_path = '/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_{}.pickle'.format(SETTING)
        saved_model = '/usr0/home/yuncheng/MultiBench/synthetic/model_selection/{}/{}_{}_best.pt'.format(SETTING, SETTING, method)
        saved_cluster = '/usr0/home/yuncheng/MultiBench/synthetic/model_selection/{}/{}_{}_cluster.pickle'.format(SETTING, SETTING, method)
        _, _, _, testdata = get_dataloader(path=data_path, keys=['0','1','label'], modalities=[1,1], batch_size=128, num_workers=4)
        model = torch.load(saved_model).cuda()
        num_params = sum([p.numel() for p in model.parameters()])
        if method in ['additive', 'agree', 'align']:
            acc = test_ensemble(model, testdata, no_robust=True, criterion=torch.nn.CrossEntropyLoss(), save_preds=saved_cluster)
        else:
            acc = test(model, testdata, no_robust=True, criterion=torch.nn.CrossEntropyLoss(), save_preds=saved_cluster)
        with open(saved_cluster, 'rb') as f:
            preds = pickle.load(f)
        with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_{}_cluster.pickle'.format(SETTING), 'rb') as f:
            cluster = pickle.load(f)
        pred_results = (cluster['test']['0'], cluster['test']['1'], preds.reshape(-1,1))
        P, maps = convert_data_to_distribution(*pred_results)
        results[method][SETTING] = get_measure(P)
        results[method][SETTING]['acc'] = acc
        results[method][SETTING]['params'] = num_params
        print()

with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/results.pickle', 'wb') as f:
    pickle.dump(results, f)

maps2
Train data: 672
Valid data: 84
Test data: 88
acc: 0.36363636363636365


/usr0/home/yuncheng/miniconda3/envs/multibench/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Redundancy 0.25493315428829044
Unique 0.24084311647191153
Unique 0.024454060329114166
Synergy 0.4022239205538314

maps2
Train data: 672
Valid data: 84
Test data: 88
acc: 0.5
Redundancy 0.10450894868078536
Unique 0.26887867229181706
Unique 7.651484056453488e-08
Synergy 0.18770946896049534



/usr0/home/yuncheng/miniconda3/envs/multibench/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


#### model selection

In [2]:
with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/results.pickle', 'rb') as f:
    results = pickle.load(f)
print(results.keys())
with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/datasets.pickle', 'rb') as f:
    datasets = pickle.load(f)
print(datasets.keys())

dict_keys(['additive', 'agree', 'align', 'early_fusion', 'elem', 'mfm', 'mi', 'mult', 'outer', 'tf', 'lower'])
dict_keys(['synthetic1', 'synthetic2', 'synthetic3', 'synthetic4', 'synthetic5', 'maps', 'maps2'])


In [5]:
with open('/usr0/home/yuncheng/MultiBench/synthetic/experiments2/datasets.pickle', 'rb') as f:
    old_datasets = pickle.load(f)
with open('/usr0/home/yuncheng/MultiBench/synthetic/experiments2/results.pickle', 'rb') as f:
    old_results = pickle.load(f)
normalized_old_datasets = dict()
for setting in old_datasets:
    normalized_old_datasets[setting] = dict()
    datasets_total = sum([old_datasets[setting][measure] for measure in MEASURES])
    for measure in MEASURES:
        normalized_old_datasets[setting][measure] = old_datasets[setting][measure] / datasets_total

In [9]:
methods = ['additive', 'agree', 'align', 'early_fusion', 'elem', 'recon', 'mi', 'mult', 'outer', 'lower']
model_selection = dict()
for setting in old_datasets:
    measures = sorted([(old_results[method][setting]['acc'], method) for method in methods])
    model_selection[setting] = measures[-3:]
# model_selection

In [11]:
normalized_results = dict()
normalized_datasets = dict()
for method in METHODS:
    normalized_results[method] = dict()
    for setting in SETTINGS:
        normalized_results[method][setting] = dict()
        normalized_datasets[setting] = dict()
        results_total = sum([results[method][setting][measure] for measure in MEASURES])
        datasets_total = sum([datasets[setting][measure] for measure in MEASURES])
        for measure in MEASURES:
            normalized_results[method][setting][measure] = results[method][setting][measure] / results_total
            normalized_datasets[setting][measure] = datasets[setting][measure] / datasets_total
accs = [results[method][setting]['acc'] for setting in SETTINGS for method in METHODS]
model_selection_best = dict()
for setting in SETTINGS:
    measures = sorted([(results[method][setting]['acc'], method) for method in METHODS])
    model_selection_best[setting] = results[measures[-1][1]][setting]['acc']
    print(setting, measures)

synthetic1 [(0.5238888888888888, 'align'), (0.5733333333333334, 'mi'), (0.5966666666666667, 'elem'), (0.6283333333333333, 'additive'), (0.6566666666666666, 'agree'), (0.6777777777777778, 'early_fusion'), (0.6811111111111111, 'outer'), (0.69, 'lower'), (0.6916666666666667, 'mult'), (0.6938888888888889, 'mfm')]
synthetic2 [(0.5033333333333333, 'elem'), (0.5122222222222222, 'mi'), (0.5338888888888889, 'outer'), (0.5477777777777778, 'lower'), (0.5527777777777778, 'mult'), (0.5611111111111111, 'agree'), (0.5611111111111111, 'align'), (0.5616666666666666, 'additive'), (0.5694444444444444, 'early_fusion'), (0.5944444444444444, 'mfm')]
synthetic3 [(0.4261111111111111, 'additive'), (0.4861111111111111, 'align'), (0.5461111111111111, 'mi'), (0.5622222222222222, 'elem'), (0.6061111111111112, 'agree'), (0.625, 'outer'), (0.6366666666666667, 'early_fusion'), (0.6366666666666667, 'lower'), (0.6472222222222223, 'mult'), (0.6555555555555556, 'mfm')]
synthetic4 [(0.6105555555555555, 'mi'), (0.612777777

In [15]:
best_model_scores = []
for setting in datasets:
    tmp = []
    for old_setting in old_datasets:
        tmp.append((np.sum(np.absolute([normalized_datasets[setting][measure]-normalized_old_datasets[old_setting][measure] for measure in MEASURES])), old_setting))
    tmp = sorted(tmp)
    for i in range(1):
        print(setting, "most similar dataset:", tmp[i], "selected models", model_selection[tmp[i][1]])
        selected_models = model_selection[tmp[i][1]]
        model_scores = []
        for _, method in selected_models:
            if method == 'recon':
                method = 'mfm'
            model_scores.append((results[method][setting]['acc'], method))
        model_scores = sorted(model_scores)
        best_score = model_scores[-1][0] / model_selection_best[setting] * 100
        best_model_scores.append(best_score)
        print('{:.2f}%'.format(best_score))
print('Overall:{:.2f}%'.format(np.mean(best_model_scores)))

synthetic1 most similar dataset: (0.14633150964283043, 'synthetic5') selected models [(0.6844444444444444, 'early_fusion'), (0.685, 'recon'), (0.6877777777777778, 'agree')]
100.00%
synthetic2 most similar dataset: (0.18355423842472463, 'synthetic3') selected models [(0.5461111111111111, 'recon'), (0.5538888888888889, 'align'), (0.555, 'mult')]
100.00%
synthetic3 most similar dataset: (0.2793375138069869, 'synthetic5') selected models [(0.6844444444444444, 'early_fusion'), (0.685, 'recon'), (0.6877777777777778, 'agree')]
100.00%
synthetic4 most similar dataset: (0.11088404258613735, 'synthetic2') selected models [(0.7138888888888889, 'additive'), (0.7183333333333334, 'align'), (0.72, 'agree')]
99.53%
synthetic5 most similar dataset: (0.14912201008589812, 'mix2') selected models [(0.7116666666666667, 'recon'), (0.7205555555555555, 'agree'), (0.7233333333333334, 'align')]
100.00%
maps most similar dataset: (0.09856786863625443, 'synthetic4') selected models [(0.6522222222222223, 'recon'),

In [4]:
for setting in datasets:
    print(setting, datasets[setting])

synthetic1 {'redundancy': 0.042766259538634416, 'unique1': 0.02596364321132929, 'unique2': 2.17157384344753e-15, 'synergy': 0.10614122901494483}
synthetic2 {'redundancy': 0.012672691701037832, 'unique1': 2.8259215299914983e-05, 'unique2': 0.0061237290331549, 'synergy': 0.05926624624675512}
synthetic3 {'redundancy': 0.027742116209452286, 'unique1': 0.025980735076023997, 'unique2': 1.4684105273299479e-15, 'synergy': 0.06713126114497113}
synthetic4 {'redundancy': 0.06557045989622268, 'unique1': 2.1518739480906317e-15, 'unique2': 0.056037809617245915, 'synergy': 0.053704370671353585}
synthetic5 {'redundancy': 0.03943597142606828, 'unique1': 0.043663108779810396, 'unique2': 4.6119695336763056e-15, 'synergy': 0.06589532206160133}
maps {'redundancy': 0.08691429284341395, 'unique1': 1.1195372689354384e-07, 'unique2': 0.08681460221097399, 'synergy': 0.2624709049842621}
maps2 {'redundancy': 0.12120173624349637, 'unique1': 0.004155439963112993, 'unique2': 0.03654196259113951, 'synergy': 0.4022303

In [11]:
if os.path.isfile('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/datasets.pickle'):
    with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/datasets.pickle', 'rb') as f:
        results = pickle.load(f)
else:
    results = dict()
with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/DATA_maps2_cluster.pickle', 'rb') as f:
    dataset = pickle.load(f)
print('maps2')
data = (dataset['test']['0'], dataset['test']['1'], dataset['test']['label'])
P, maps = convert_data_to_distribution(*data)
result = get_measure(P)
results['maps2'] = result     

with open('/usr0/home/yuncheng/MultiBench/synthetic/model_selection/datasets.pickle', 'wb') as f:
    pickle.dump(results, f)

maps2
Redundancy 0.12120173624349637
Unique 0.004155439963112993
Unique 0.03654196259113951
Synergy 0.40223033982849343


Format results in $\LaTeX$

In [16]:
for setting in ['maps', 'maps2']:
    # for method in ['early_fusion', 'additive', 'agree', 'align']:
    # for method in ['elem', 'outer', 'mi', 'mult']:
    for method in ['lower', 'mfm']:
        for measure in MEASURES:
            print('&', '${:.2f}$'.format(results[method][setting][measure]), end=' ')
        print('&', '${:.2f}$'.format(results[method][setting]['acc']), end=' ')
    print('\\\\')

& $0.53$ & $0.04$ & $0.04$ & $0.09$ & $0.41$ & $0.28$ & $0.06$ & $0.07$ & $0.10$ & $0.56$ \\
& $0.12$ & $0.24$ & $0.00$ & $0.22$ & $0.51$ & $0.23$ & $0.17$ & $0.00$ & $0.18$ & $0.52$ \\
